In [1]:
%config InlineBackend.figure_formats = ['svg']
import sys
sys.path.insert(1, '../')

import matplotlib.pyplot as plt
from os import path

from deployment_instance import ExperimentResult
from src.ingest_data import ingest_experiment_results
from src.query_data import get_exfiltration_time_df, get_data_exfiltration_cdf
from statistics import mean
import seaborn as sns
# set seaborn font size
sns.set_theme(font_scale=1.5)
sns.set_style("whitegrid", {'axes.grid' : False})

import pandas as pd

In [2]:
def get_number_files_exfiltrated(data: dict[str, ExperimentResult], expected_files: int):
    df = pd.DataFrame(
        columns=[
            "experiment_type",
            "id",
            "files_exfiltrated",
        ]
    )

    for experiment_num, experiment_result in data.items():
        files_exfiltrated = len(experiment_result.data_exfiltrated)
        df.loc[df.shape[0]] = [
            experiment_result.scenario.name,
            experiment_num,
            files_exfiltrated,
        ]

    return df

In [3]:
def convert_data_to_df(data: dict[str, ExperimentResult], experiment: str, num_expected_files):
    df = pd.DataFrame(
        columns=[
            "experiment",
            "type",
            "name",
            "experiment_num",
            "time_exfiltrated",
            "file_number",
            "percent_data",
        ]
    )

    for experiment_num, experiment_result in enumerate(list(data.values())):
        for idx, data_exfiltrated in enumerate(experiment_result.data_exfiltrated):
            if 'reactive' in experiment_result.scenario.name:
                exp_type = 'reactive'
            else:
                exp_type = 'non-reactive'

            df.loc[df.shape[0]] = [
                experiment,
                exp_type,
                experiment_result.scenario.name,
                experiment_num,
                data_exfiltrated.time_exfiltrated / 60,
                idx + 1,
                ((idx + 1) / num_expected_files) * 100,
            ]

    return df

In [4]:
# Equifax Medium
baseline_reactiveStandalone_d0_h50 = ingest_experiment_results(path.join('data/Reactive/EquifaxMedium/baseline_reactiveStandalone_d0_h50'))
baseline_reactiveStandalone_d10_h0 = ingest_experiment_results(path.join('data/Reactive/EquifaxMedium/baseline_reactiveStandalone_d10_h0'))
baseline_reactiveStandalone_d10_h50 = ingest_experiment_results(path.join('data/Reactive/EquifaxMedium/baseline_reactiveStandalone_d10_h50'))
baseline_reactiveLayered_d10_h50 = ingest_experiment_results(path.join('data/Reactive/EquifaxMedium/baseline_reactiveLayered_d10_h50'))

medium_data = {}

medium_data.update(baseline_reactiveStandalone_d0_h50)
medium_data.update(baseline_reactiveStandalone_d10_h0)
medium_data.update(baseline_reactiveStandalone_d10_h50)
medium_data.update(baseline_reactiveLayered_d10_h50)

medium_times = get_exfiltration_time_df(medium_data, 24)
medium_times.groupby('experiment').agg({'time_per_file': ['mean', 'std'], 'percent_files_exfiltrated': ['mean', 'std']})

FileNotFoundError: [Errno 2] No such file or directory: 'data/Reactive/EquifaxMedium/baseline_reactiveStandalone_d0_h50'

In [15]:
# Equifax Small
baseline_reactiveStandalone_d0_h50 = ingest_experiment_results(path.join('data/Reactive/EquifaxSmall/baseline_reactiveStandalone_d0_h50'))
baseline_reactiveStandalone_d10_h0 = ingest_experiment_results(path.join('data/Reactive/EquifaxSmall/baseline_reactiveStandalone_d10_h0'))
baseline_reactiveStandalone_d10_h50 = ingest_experiment_results(path.join('data/Reactive/EquifaxSmall/baseline_reactiveStandalone_d10_h50'))
baseline_reactiveLayered_d10_h50 = ingest_experiment_results(path.join('data/Reactive/EquifaxSmall/baseline_reactiveLayered_d10_h50'))

small_data = {}

small_data.update(baseline_reactiveStandalone_d0_h50)
small_data.update(baseline_reactiveStandalone_d10_h0)
small_data.update(baseline_reactiveStandalone_d10_h50)
small_data.update(baseline_reactiveLayered_d10_h50)

small_times = get_exfiltration_time_df(small_data, 12)
small_times.groupby('experiment').agg({'time_per_file': ['mean', 'std'], 'percent_files_exfiltrated': ['mean', 'std']})

time_per_file            \
                                             mean       std   
experiment                                                    
baseline_reactiveStandalone_d0_h50       0.773935  1.068749   
baseline_reactiveStandalone_d10_h0       0.883195  0.072994   
baseline_reactiveStandalone_d10_h50      0.744059  1.108372   

                                    percent_files_exfiltrated             
                                                         mean        std  
experiment                                                                
baseline_reactiveStandalone_d0_h50                   8.333333  11.111111  
baseline_reactiveStandalone_d10_h0                 100.000000   0.000000  
baseline_reactiveStandalone_d10_h50                  6.666667   9.460770

In [13]:
get_exfiltration_time_df(baseline_reactiveStandalone_d10_h0,12)

,experiment,experiment_num,time_exfiltrated,time_per_file,files_exfiltrated,percent_files_exfiltrated
0,baseline_reactiveStandalone_d10_h0,0,10.367174,0.863931,12,100.000000
1,baseline_reactiveStandalone_d10_h0,1,10.542728,0.878561,12,100.000000
2,baseline_reactiveStandalone_d10_h0,2,10.005792,0.833816,12,100.000000
3,baseline_reactiveStandalone_d10_h0,3,12.360477,1.030040,12,100.000000
4,baseline_reactiveStandalone_d10_h0,4,11.189626,0.932469,12,100.000000
5,baseline_reactiveStandalone_d10_h0,5,10.701017,0.891751,12,100.000000
6,baseline_reactiveStandalone_d10_h0,6,7.261210,0.907651,8,66.666667
7,baseline_reactiveStandalone_d10_h0,7,10.937793,0.911483,12,100.000000
8,baseline_reactiveStandalone_d10_h0,8,9.244059,0.770338,12,100.000000
9,baseline_reactiveStandalone_d10_h0,9,10.036343,0.836362,12,100.000000


In [14]:
baseline_reactiveStandalone_d10_h0

{'2024-10-22_00-01-24': ExperimentResult(scenario=Scenario(name='baseline_reactiveStandalone_d10_h0', defender=DefenderInformation(telemetry='ReactiveCredentials', strategy='ReactiveStandalone', capabilities={'DeployDecoy': 10, 'HoneyService': 0, 'RestoreServer': -1, 'HoneyCredentials': 0}), attacker=AttackerInformation(name='equifax_baseline'), deployment_instance=DeploymentInstanceInformation(name='EquifaxSmall')), experiment_time=1046.2054288387299, execution_time=629.1674067974091, setup_time=417.0380220413208, flags_captured=[], data_exfiltrated=[DataExfiltrated(name='data_database_3.json', time_exfiltrated=151.0304672718048), DataExfiltrated(name='data_database_5.json', time_exfiltrated=204.0304672718048), DataExfiltrated(name='data_database_1.json', time_exfiltrated=251.0304672718048), DataExfiltrated(name='data_database_9.json', time_exfiltrated=288.0304672718048), DataExfiltrated(name='data_database_6.json', time_exfiltrated=331.0304672718048), DataExfiltrated(name='data_datab